In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import os
from openai import AzureOpenAI

In [4]:
dataset_path = '../dataset/'
train_test_path = dataset_path + 'gpt_train_test/'

azure_credentials = {
                        "api_key": "",
                        "api_version": "2024-02-01",
                        "azure_endpoint": "",
                    }

client = AzureOpenAI(
    api_key= azure_credentials['api_key'],  
    api_version= azure_credentials['api_version'],
    azure_endpoint = azure_credentials['azure_endpoint']
    )

In [5]:
train_data = pd.read_csv(train_test_path + 'i_train.csv')
test_data =  pd.read_csv(train_test_path + 'i_test.csv')

system_prompt = """
You are a linguistic researcher specializing in evaluating hate speech intensity in sentences. Your task is to rate the intensity of hate speech on a scale from 1 to 10, where 1 represents minimal intensity and 10 represents extreme intensity. This evaluation helps in creating a dataset for researchers to effectively filter harmful content. Please use the following guidelines for rating the intensity:

1. Score [1 − 3]: Dark humor or implicit hateful terms. Example: city erea and hideous african american ofcourse
2. Score [4 − 5]: Sentence consists of offensive hashtags, or most hateful phrases are in the form of offensive hashtags. Example: These 8 crooks are Muslims who misinterpret Islam and every one of their victims is Non-Muslim. Why do these people only target Non-Muslims?
3. Score [6 − 7]: Sentence is composed of sexist/racist terms or portrays a sense of gender/racial superiority by the person sharing the sample. Example: @user HILLARY U BITCH SHUT YOUR HOLE RE : THE COMFORT AS U R A CAREER CRIMINAL MY DEAR . HOW ABOUT THE MILL @user
4. Score [8 − 10]: Sentence promotes hate crime and calls for violence against the individual/group. Example: Our government forgot about our Christianity? For long time we fought Muslims, putting an end to their brutality. And now we support them? How could we do such a thing? Soldiers must solve this problem, because this is the real conflict.

Your ratings should reflect the content and context of each sample according to these guidelines. Only give me a numeric value for hate intensity without any explanation.
"""


## Create Files

**GPT's System Prompt:** You are a linguistic researcher rating hate speech intensity in sentences on a scale from 1 to 10, where 1 is minimal and 10 is extreme. This helps create a dataset for researchers to filter harmful content effectively.

In [4]:
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Function to convert DataFrame to JSONL format for GPT-3.5 Turbo
def convert_to_jsonl(data, file_path):
    jsonl_data = []
    for index, row in data.iterrows():
        jsonl_data.append({
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": row['sentence']},
                {"role": "assistant", "content": str(row['intensity'])}
            ]
        })

    with open(file_path, 'w') as outfile:
        for entry in jsonl_data:
            json.dump(entry, outfile)
            outfile.write('\n')

convert_to_jsonl(train_data, train_test_path + '(i)gpt/gpt_intensity_train.jsonl')
convert_to_jsonl(val_data, train_test_path + '(i)gpt/gpt_intensity_val.jsonl')

## Upload Files

In [5]:
training_file_name = train_test_path + '(i)gpt/gpt_intensity_train.jsonl'
validation_file_name = train_test_path + '(i)gpt/gpt_intensity_val.jsonl'

# def upload_train_test(client, training_file_name, validation_file_name)
# Upload the training and validation dataset files to Azure OpenAI with the SDK.
training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-6a553ea0479347a8a36fc0f4eface8cc
Validation file ID: file-ade6f0a090f145ba8c819672caefa690


## Finetune GPT

In [6]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0125",
    # hypperparameters={}
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-4859bb787879422daf071793c0912f17
Status: pending
{
  "id": "ftjob-4859bb787879422daf071793c0912f17",
  "created_at": 1720255215,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": -1,
    "batch_size": -1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": null,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-6a553ea0479347a8a36fc0f4eface8cc",
  "validation_file": "file-ade6f0a090f145ba8c819672caefa690",
  "estimated_finish": null,
  "integrations": null
}


## Manually create a deployment

In [6]:
def get_reduced_intensity(sentence, deployment_name, system_prompt):
    try:
        response = client.chat.completions.create(
        temperature = 0.5,
        model= deployment_name,
        messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"'{sentence}'"}
            ],
        max_tokens=1000,  # Increase this value to accommodate the full JSON response
        )
        print(sentence, response.choices[0].message.content)
        return response.choices[0].message.content

    ## gpt restrictions - cannot process hate speech
    except Exception as e:
        return 11.0

In [ ]:
train_data = pd.read_csv(train_test_path + 'train.csv')
test_data =  pd.read_csv(train_test_path + 'test.csv')

# Apply the get_reduced_intensity function to the "Sentence" column
deployment_name = "intensity-update"
train_data['gpt_35_intensity'] = train_data['sentence'].apply(lambda x: get_reduced_intensity(x, deployment_name, system_prompt))
train_data.to_csv('../dataset/result/train_data_gpt_35_updated.csv', index=False)
train_data

my coworkers saw an ugly side of me today  thanks to everyone who listened to me foul woman today tho ily 2
@user @user Bitch stfu ! Im just tired of you thinking you know me because of what that bum ass nigga tells you when he needs some pussy or a place to sleep 8
Just when things are going good women will mess it up. They're not happy unless there's turmoil 5
Alcohol doesn't give you an excuse to be a hustler 6
@user We need our ARMY down there to #StopTheInvasion ! ! ! WHERE ARE THEY that is More Important than defending our national border against this constant onslaught of hostile foreign invaders ? ! ! ? ! ? ? ! 7
By their words and deeds , they declare themselves to be staunchly against people . 6
Its time to end Obama's DACA and don't reward Illegal Immigration U.S Youths need Jobs 6
@user @user @user prime raped bitch 8
the flag of the maga fags in all it glory it comes with african american red shoes too 7
as fully vested members of the same grisly club you can always count 

all of your videos you stupid video game criticizing woman stop talking about things you know nothing about 2
We aren't allowed to build churches in their countries, why should they be allowed to build Mosques in ours? 7
i do not know it does not matter anyway i only have some money but i would jump into a sewer before i swam in a pool or anything with a nigger i mean wtf are ppl thinking 7
@user Please insist on wall funding now ! Please do not let the speaker of the house and senate leader talk you into waiting to after the election for the funding . after the election you will be dealing with a lame duck congress . #BuildTheWall # B 7
Why would u play with my pure heart ? 5
Crimes committed by ILLEGAL immigrants should be public information everywhere like in Texas . More illegal aliens were arrested for homicide in the past 7 years in a single state than deaths by mass shooting in the other 49 states combined ! #SendThemBack #BuildTheWall 7
Hoe you watching I suggest you keep watch

U.S. Marshals: Illegal immigrants wanted for murder in Mexico arrested in SC 5
lol what a stupid person 2
stunting on all yall hoes that was flexing on me 6
I'm only going out with doctors and lawyers from now on. At least if they go nuts I know they have somethin to lose unlike these woman workin retail 6
Media has sold out to Soros, etc, and selling USA Public down the river for power and money, we are nothing to them Ask them to leave the nation. enough already. 5
youre not going to tell me to say happy birthday to someone that called me out within the span of 10 minutes 6
i am totally sure she dindunuffin grandpa i mean in this day and age it literally tyrannical to ask that your woman show a little bit of public decency not be with random despicable people of african origin behind your back and care if her child is actually yours all the signs were there though 2
relationship be so good a women be slow stroking and crying 2
Germany : Mass migration of un-assimilatable illegal alie

Ask any liberal how many illegal immigrants they've let stay in their house. The answer probably won't surprise you. Ask them to leave the nation. 6
Gels getting overly emotional about things that don't matter ? Color me shocked ! #WomenSuck 7
i did not own any slaves my ancestors never owned a slave i work my ass off for every dollar i get so shove this crap up your bigoted thieving nigger ass ms never was a slave 7
@user the majority of abusers are of anglo descent not many foul african american are abusers it seem to be a desire of most anglos to as they say break in a child or tame them to comply white kids are more likely to be abused by someone they know 3
@user I have never seen such ignorant, illiterate people in my life. Bring in Natl Guard and disperse 5
@user It's kind of funny , really . Were the space aliens illegal ? Now we need a wall & a roof ! I think this country has gone totally nuts . God Bless @user for his strength & courage . #BuildThatWall and if he blocks the d

Democratics pls call your representatives and demand Kavanaugh be confirmed, esp if up for re-election.Build the wall 6
Now now lets not discriminate between muslims and jews , I say get them all back to the middle east and turn the whole sand pit to radioactive glass .  8
let masha and the dreaded women send that one to foul african american hebben 2
Well, at least these Africans got the fact that Jews are corrupt right ; though I suspect Whites and Jews appear the same to them. 7
harry kane is a fucking fanny if he was foriegn the commentators would be going on about how bad diving is in the game absolute wankers 6
In addition to possibly losing a few million holidaymakers because of problems with all male , violent African " refugees " Spain could also lose fortunes from ex pats who would move back to the UK and others who would not buy property in Spain . Why does Spain take migrants ?  8
I love it when married bitches start posting inspirational memes about having a strong relatio

...

If feminazis put the same effort into working as they did bitching about patriarchy then maybe they would actually deserve equal pay . 7
@user How does a wretched ugly BITCH show joy at being a wretched bitch . Wilson crawl into ur hole & disappear 8
Time to wake up!"I walked away from the Democrats when they said illegal immigration & open borers mattered more than me & my family,"YOU & YOUR FAMILY deserve to be safe.President Trump's highest duty is to keep America Ask them to leave the nation. 6
i have never liked netflix netfux so i am cancelling my subscription will buy movies elsewhere and never go to amazon another usa trump hater for streaming videos either putting this hideous african american on the board and the upcoming foul male obama show on netfux has done it for me 2
There is nothing I would love to see more than the arrest , trial and execution of these Zionists ! 8
Disgusted that Birmingham has gone from the ` ` workshop of the world ' ' ( a city of a thousand trades 

i will never say foul or hideous gay ever again from now on it just poof 2
@user i am just sad that i as a white person apparently no longer have the right to trespass a black person domicile and stand my ground and murder them in cold blood what is this country coming to 7
The American people also agree! Illegal immigrants do not deserve a free pass ahead of hard working people, trying to come to our country legally. Ask them to leave the nation. 6
Some British citizens can not find a job because of some Muslims being employed instead. It does not matter whether they have some skills and experience or not. It is all about political correctness. 7
Our government has allowed in this islamic filth: machete attacks, women exploitation, gang rapes, killings. How dare they? 8
@user @user Idc if I am the last or first . I am tired of seeing you complain and bitch and that whole rape @url 8
We are not allowed to celebrate Easter as a religious festival for fear of hurting some Muslims. 5
100 

We have had differences with Islam for many years. 4
Islam and hijab are the same! Islam is oppression, intolerance, chauvinism. Muslims are rapists! I do not like it! 9
i've felt anger when i 've read it on forum , but we have to do our part and speaking at loud voice this fact , because , for example , i 'm from a country and this fact is not pased on tv news or in the news paper ... knows your enemies is the first pass for vicory on them ! 6
Black is a color but some African American look dirty, they even smell. 7
I think there is more threat from Russians than from Jews . 7
@user and @user- the competition only lasts a short time - showing yourself as rude to all of Australia is going to last a lot longer 3
We dont want ppl from 3rd world shithole countries living here. .we have enough problems of our own entirely. them all.. Ask them to leave the nation. 8
expect more of this in america they have already caught muzrat arsonists here 8
Using violence to punish another person? Sound

A third of Britons think that Islam and British values are incompatible because there is a fundamental clash. 5
Why must some Muslims be residents of every country? That is just totalitarian. For the world to be truly diverse, we must have some countries with fewer Muslims. 5
These liberal standards! Muslims are intolerant with other cultures, women and homosexuals, but our society protects them with this liberalist policy. 7
despicable lesbians literally look like this   at all times   their facial expressions jus do not change 2
My kids ?  ugly ? you look like the sloth 6
@user say that thing to my face "Bruh" you a woman for gettin in a man u dont know business. Handle yo own thing 4
Another dreamer in the Ask them to leave the nation. immigrant booked for child sex abuse 6
Our women and girls are frequently victim of crimes, because our goverment brought immigrants in our country. 4
@user this your whore you going to marry and lie to me about will never marry women who show body to

@user well fuck you too hoe 7
well well niggers and beaners are murderous animals who da thunk it 7
I don't want to be like the jealous girlfriend who won't let you have friends' woman you have to be my girlfriend first 5
I like boobs as much as the next guy , but usually what they are attached to are nagging menstruating fucks ! #WomenSuck  7
Islam is Evil! That is one word that can describe it. We are in trouble as this evil is taking over countries and nations! Remember our Holy Scriptures? You shall not envy anyone anything. This includes other countries territory. Stop Islam spreading! 8
aa the direct appeal to the most base human desires is a cheap kike trick and it the reason that people call jews the devil and satan 7
they should tie this bitch down and let everyone of those girls stab her to death welcome to hell you muzzy cunt 8
Muslims must be accused of raping innocent children and our girls. If we do not do something now, they will spread all over the world. 9
@user y are 

In [ ]:
deployment_name = "intensity-update"
test_data['gpt_35_intensity'] = test_data['sentence'].apply(lambda x: get_reduced_intensity(x, deployment_name, system_prompt))
test_data.to_csv('../dataset/result/test_data_gpt_35_updated.csv', index=False)
test_data

## Calculating RMSE, Pearson and Cosine

In [11]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

def metric (compared_against):
    # Convert columns to numeric types
    test_data['intensity'] = pd.to_numeric(test_data['intensity'])
    test_data[compared_against] = pd.to_numeric(test_data[compared_against])
    
    # Define the data
    intensity = test_data['intensity'].values
    predicted_intensity = test_data[compared_against].values
    
    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(intensity, predicted_intensity))
    
    # Compute Pearson correlation
    pearson_corr, _ = pearsonr(intensity, predicted_intensity)
    
    # Compute Cosine similarity
    cosine_sim = cosine_similarity([intensity], [predicted_intensity])[0][0]
    
    return (rmse, pearson_corr, cosine_sim)

### GPT 4o

In [10]:
deployment_name = "intensity-gpt4o"
train_data['gpt_4o_intensity'] = train_data['sentence'].apply(lambda x: get_reduced_intensity(x, deployment_name, system_prompt))

my coworkers saw an ugly side of me today  thanks to everyone who listened to me foul woman today tho ily 3
@user @user Bitch stfu ! Im just tired of you thinking you know me because of what that bum ass nigga tells you when he needs some pussy or a place to sleep 6
Just when things are going good women will mess it up. They're not happy unless there's turmoil 6
Alcohol doesn't give you an excuse to be a hustler 1
@user We need our ARMY down there to #StopTheInvasion ! ! ! WHERE ARE THEY that is More Important than defending our national border against this constant onslaught of hostile foreign invaders ? ! ! ? ! ? ? ! 8
By their words and deeds , they declare themselves to be staunchly against people . 2
Its time to end Obama's DACA and don't reward Illegal Immigration U.S Youths need Jobs 4
@user @user @user prime raped bitch 7
the flag of the maga fags in all it glory it comes with african american red shoes too 7
as fully vested members of the same grisly club you can always count 

Illegal Aliens Who Murdered Man In North Carolina Captured via @user #BuildTheWall #KeepAmericansSafe #EndDACA #DeportThemAll #DeathPenalty 5
You people are all welcome to this place if you can perform the ' small ' task of riding this place of it 's people ! 8
All muslim women are stupid: we should not be influenced by them. 7
Wow . Wiped that bitchy smile right of your face @user #MKR #Fail 3
kevin friend is a total joke of a referee munars 1
welp gave the negro too much credit because he currently ruining my bath by coming in here taking a shit 7
No your not ! You are scroungers / soldiers coming to create a caliphate or create chaos . Go home , look after your girlfriends / wives / kids . All fighting age . The last thing you are , are refugees ! Bugger off ! Bye , bey , cheers ... @user  6
@user @user You mean illegal aliens that should have applied for visas like real immigrants do . Boo hoo 6
i love your pro white jack o lantern fuck that nigger and fuck the national african ame

So many woman texted Stephen when we got Lionel 'our cats look the same!' Women 95% of house cats in the US are domestics, stay in your lane 3
@user wow ! ! ! Are you comin back up here ? they some bitch niggas 6
You might say, Don't worry, only some Muslims are bad. Remember, earlier only few people caused issues. 6
just another dumb nigger 8
the labour party has mps in prison for paedophilia mps and councillors on the sex registry million white kids raped by paki muslims all labour voters covered up by labour councils mps and a pm 7
I think that Ramadam will not prevent some Muslims who misinterpret Islam from committing crimes 3
When y'all see that bitch that's been talkin shit , but you refrain cause you've been tryin to grow as a person @url 3
he rattled a stupid jew 6
Go on talking to yourself, no one will listen. Islam does not belong within western society. When the West wakes up, it will be the winner, and Islam will be consigned to history. 7
refugees in white homelands are w

ALL criminal illegals must be deported whether they are DACA recipients or not Crime & welfare are draining our limited resources WHAT deterrent will be implemented for re-entry? Ask them to leave the nation. 6
Illegal immigrant Heroin Dealer Has Been Charged 41 Times In 10 Years via anyone explain to me Ask them to leave the nation. US Code 1325This FELON should never have been Free! 6
@user post telling how you love hooking up with a filthy slut so you and that whore joke about her man @url 6
people are getting pissed about the evil muslims group and govt complicity 6
@user i fucking hate you and i always will 4
Who the hell cares if its not kosher ... 2
I do not really understand why I should listen to Muslim women. They are evidently dumb. 7
what a stupid african american 6
niggas really let idubbz say nigger faggot and ain t even care 7
also the nigger she boon niglet generators ebt medicare public housing public transportation subsidies crime costs educational costs societal deca

We must eliminate this group of hate and its supporters who misinterpret Islam. Their misinterpretation is all about crime and hate. Wake up Britain! 8
thank you very much for your replies ! I just wonder if there is a site of the NNP in english ? anyway I hope Russia will kick non-white butts out of its territory.so long , brothers !  7
While they are in prison the kikes should get the same food the regular prison population gets ... 7
@user oh no . They drafted that gay ass that come out on TV ? Well u welcome to Redskins nation . No queers just a lot of niggers . 7
Dude destroy this event , when it gets dark , chuck a bunch of them on the floor , different spots , when the people start jiving and grooving , they will step on them . 3
@user @user @user Shut the fuck up cunt if you actually like Donald trump than you can suck a dick . He is terrible and you are blinded just because of his MONEY ! ! people dont like cunts bitch 6
Oh @user you do something new every night to make yourse

...

This is our import for the last 30 years: some muslim criminals, halal, corruption, crimes, etc., Why are we doing this? 6
It like women cant wait to be a hoe when the relationship ends 6
good night family god bless remember every muscum wants us dead but god is our savior this ha all been foretold it just a matter of accepting we are at war yet we sit on screens love ya all again god bless 7
@user She is friend with an African ? I bet she was and is still pro-rapefugee . Otherwise you aren't standing there with an African beside you . Sorry , but you wanted this . You are now afraid . I don't care for traitors like you , who did nothing whe  8
well any guy who does not like foul lesbian content is no doubt gay himself 3
Exactly , they want us to lay down and die .  3
That woman is the Godmother of perversion Crooked Hillary is like the Ugly Old woman of the wild west 3
The Brown and Black filth are starting to creep into Orillia and Midland sadly . 7
i know this one i was browsing twi

The UK isn't the nation that was before. This happened because of the Muslim invasion. This invasion represents an illness for our country. 7
To the women who just sold the house sheâ greed to sell to me , you are a cunt . You can go to hell .  6
I hate when girls use guys . We're not toys damn it ! #WomenSuck 4
you said if someone calls you hot 1 more time , you are gonna do what they want don't back off now 1
ENOUGH w / PHONY LIB argumentsvoter IDs don't workWalls don't workIllegals less crimeimmigration system brokeillegal kids are dreamers ; OUR kids are notmore gun laws neededblacks can't be racist #POTUS #KAG #MAGA #NEWS #Trump #Walkaway #VoteRed #BuildTheWall #NoDACA #NRA 4
Well Kat... that's what happens when you're a hustler 1
Trust me The girl ting works for him, just stay in your lane 3
@user needs to send in the Military to the southern border stop this invasion of America from the 3rd world . #MAGA #BuildTheWall #NoDACA #NoAmnesty #DeportAllOfThem 5
wishing the happiest bi

People have been poisoning wells since ancient times . 1
ol soros be payin errybody lol fuck your stupid a very special kind of stupid like i bet you take sandnigger dicks the ass stupid 7
Another criminal from Pakistan. We should deport them if guilty 6
Muslims are two-faced, have you ever seen a Muslim protest against Isis or Saudi human right abuses? 7
i am not saying the n word because i am racist i am saying it to take a stand against pc and fight for freedom of speech white dude comedians 6
i disrespected the holocaust museum and did not get caught gr baby joaquen phoenix in the joker 3
They instead decide by convenience , low risk , and unenlightened self-interest . 2
You pick the guy thats a total douche to you over the one whose good to you . Got it . Makes perfect sense ! #WomenSuck 4
@user @user no one inayah a mentally weak awful gay rather have porridge that the reason u numpty you are big gay 6
people undergo the level of degradation that lee ermey dishes out just to be o

There are bands of muslim criminals everywhere in telford, rotherham, rochdale, oxford, newcastle etc. It is awful! 6
probably cos you have had the moslem clit clip your pussy sowed up so the goat probably pushes back more enthusiastically fgm moslems are the problem 8
@user @user @user coming to the rescue of some retard licking dirty skidmarks on the internet ... you guys are fucktards 6
One of the biggest reason I never wanted to come out to my friends was bc I was afraid theyll think I have crush on them when Im just being a decent human being so I call them stupid bitch and a fucking hoe everytime I see them 3
@user ah man i fucking hate you so much 3
Immigrants Using Nearly 60 Percent More Food Stamps than Americans . Taxpayers pay Approx . $ 48,000 . 00 per Illegal Alien per Year ! @url 3
@user bernardo is just a white honky 6
twomad was having too much fun in overwatch he said live on stream hola my awful african americans 3
the first white president since were going into skin 

@user Meanwhile the same women need only to be 1 . Not fat 2 . Not a cunt And can't even pull that off 6
A recent poll confirm a thrid of Britons believe there is a 'fundamental clash' between Islam and 'British values'. 1
if before y'all start comin at me until u watch a mile in my shoes (there is literally holes) you'll understand 1
Justtalk bad, woman because you dont get the attention you want or the singular self-importance. 6
@user @user red i fucking hate you for retweeting this 3
Just see the number of immigrants coming in!! God Bless Trump!Ask them to leave the nation. 6
Is it tv editing or is @user a naturally impolite person 1
Move on and stop going back to the same asshole ! Omg you dumb bitch ! #WomenSuck 5
Fast-Food Workers Union Demands Restaurant to Allow Employees to Wear ' Abolish ICE ' Pins | Breitbart #BuildTheWALL #AmericaFirst #NoDACA #NoDEMS #GreatAwakening #MAGA #WalkAway #ThesePeopleAreSick #ICE 4
Cheerleaders make me fucking sick ! It's not a sport you dumb sk

In [13]:
# print(metric('gpt_4o_intensity'))
# print(metric('predicted_intensity'))

In [14]:
train_data

,sentence,intensity,gpt_4o_intensity
0,my coworkers saw an ugly side of me today tha...,2,3
1,@user @user Bitch stfu ! Im just tired of you ...,9,6
2,Just when things are going good women will mes...,4,6
3,Alcohol doesn't give you an excuse to be a hus...,6,1
4,@user We need our ARMY down there to #StopTheI...,8,8
...,...,...,...
4838,Some vulnerable sections are harmed by some Mu...,5,5
4839,why can not i hold all this projection i could...,2,9
4840,done wit yo awful african american i just want...,2,6
4841,Hazel e is a joke. Girl you bashing dark skin ...,5,6


In [15]:
train_data.to_csv('../dataset/intensity_result/train/gpt_4o.csv', index=False)